--- Kheire --- 

Work in progress. 

Data Cleaning Phase 1 is ready. 

Data Cleaning Phase 2 is work in progress. It is done on the scraped data. 

This Notebook illustrates the different steps taken to do the data cleaning and feature engineering.

In [1]:
# import necessary libraries
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from utils import *
import re

# Data Cleaning Phase 1

This the first phase of data cleaning. It is very basic and is done on the data format and columns names. 

The first thing noticed was that when reading the original csv file "books.csv" some contents of sepicific sections in specific rows were separated by commas. This caused an error, because pd.read_csv considered them as content belonging to different sections (i.e. columns), resulting in different number of columns for several rows. Luckily, these content were separate by a comma and space ", ", which made it easy to solve the problem by replacing the ", " by "/" as seen below. "books_updated.csv" is the new updated csv which is read by pandas with success.

In [3]:
### COMMENT THE FOLLOWING CELL IF YOU HAVE ALREADY RUN IT BEFORE AND SAVED THE books_updated.csv ###
# Specify the filename
filename = 'books.csv'
updated_filename = "books_updated.csv"

## In some cases in certain fields such us author the authors are separated by ", " 
## to avoid problems when reading the csv directly using pandas and to retain all rows the following steps done

# Open the file and read lines
with open(filename, 'r', encoding='utf-8') as file: # utf-8 encoding to support all languages since there are non-english content
    lines = file.readlines()

# Process the lines to handle unwanted delimiters
cleaned_lines = []
is_firstline = True # used to avoid updating the first line

for line in lines:
    
    if is_firstline: # if it is the first line i.e. Header do not update it
        is_firstline = False
        cleaned_lines.append(line)
        continue
    cleaned_line = line.replace(', ', '/')

    # Append the cleaned line
    cleaned_lines.append(cleaned_line)

# rejoin all the lines to create a new updated csv
with open(updated_filename, 'w', encoding='utf-8') as output_file:
    for cleaned_line in cleaned_lines:
        output_file.write(cleaned_line + '\n')


In [4]:
# Read the cleaned data into a DataFrame
df = pd.read_csv(updated_filename, delimiter = ",", encoding = 'utf-8', index_col=False) 

# Look at the first 5 rows of the DataFrame
df.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic


Looking closer at the column names, we notice that in one of the column '  num_pages' the name starts with space. For ease of use, it is better to remove this unnecessary space.

In [5]:
df.columns

Index(['bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13',
       'language_code', '  num_pages', 'ratings_count', 'text_reviews_count',
       'publication_date', 'publisher'],
      dtype='object')

In [6]:
# Remove the space before the column num_pages, for ease of use
df.rename(columns={'  num_pages': '  num_pages'.replace(' ', '')}, inplace=True)
df.columns

Index(['bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13',
       'language_code', 'num_pages', 'ratings_count', 'text_reviews_count',
       'publication_date', 'publisher'],
      dtype='object')

Examining the column types, it is noticed that some of the columns that are supposed to be numerical types are object types

In [7]:
types_columns=df.dtypes
number_lines,number_columns=df.shape
print(types_columns)

bookID                 object
title                  object
authors                object
average_rating        float64
isbn                   object
isbn13                 object
language_code          object
num_pages               int64
ratings_count           int64
text_reviews_count     object
publication_date       object
publisher              object
dtype: object


In [8]:
# investigate if their are non-numerical non-date types in supposedly numerical and date columns

# text_reviews_count 
# Filter the DataFrame to get non-numerical values in the specified column
def check_non_numerical_date(columns_list: list, intended_type: str):
    
    if intended_type == "numerical":
        for column in columns_list:
            non_numerical_values = df.loc[~df[column].apply(pd.to_numeric, errors='coerce').notna()]
            print("The non_numerical_values in column {} : ".format(column), non_numerical_values[column])

    elif intended_type == "date":
        for column in columns_list:
            non_numerical_values = df.loc[~df[column].apply(pd.to_datetime, errors='coerce').notna()]
            print("The non_date_values in column {} : ".format(column), non_numerical_values[column])
    
# Display the non-numerical and non_date values of the selected columns
columns = ["isbn", "isbn13", "text_reviews_count"]
check_non_numerical_date(columns, "numerical")
check_non_numerical_date(["publication_date"], "date")

The non_numerical_values in column isbn :  3        043965548X
12       076790818X
16       076790382X
27       097669400X
40       006076273X
            ...    
11090    030727411X
11101    074347788X
11106    057305133X
11110    843221728X
11115    972233168X
Name: isbn, Length: 985, dtype: object
The non_numerical_values in column isbn13 :  1847    en-US
Name: isbn13, dtype: object
The non_numerical_values in column text_reviews_count :  1847    9/1/2003
Name: text_reviews_count, dtype: object
The non_date_values in column publication_date :  1847      MTV Books
8180     11/31/2000
11098     6/31/1982
Name: publication_date, dtype: object


From the above investigation one can notice the following : 
- isbn is not numerical since some of the isbn codes end with X 
- we notice that at index location 1847 the data is shifted to the left
- dates at index locations 8180 and 11098 were not considered as dates since they are wrong. June and November are 31 days, but in reality they are only 30 days.

Also, as seen below there is one empty cell in publisher column, which is on index location 1847.

In [9]:
df.isna().sum()

bookID                0
title                 0
authors               0
average_rating        0
isbn                  0
isbn13                0
language_code         0
num_pages             0
ratings_count         0
text_reviews_count    0
publication_date      0
publisher             1
dtype: int64

In [10]:
df["publisher"][df["publisher"].isna()==True]

1847    NaN
Name: publisher, dtype: object

In [11]:
df.iloc[1847]

bookID                6549/ said the shotgun to the head.
title                                       Saul Williams
authors                                              4.22
average_rating                                743470796.0
isbn                                        9780743470797
isbn13                                              en-US
language_code                                         192
num_pages                                            2762
ratings_count                                         214
text_reviews_count                               9/1/2003
publication_date                                MTV Books
publisher                                             NaN
Name: 1847, dtype: object

One can notice that there is a / in the bookID which separates the BookID from the book title. This must be a result from updating automatically the csv files at the beginning. 

Below is the code to fix this unwanted mistake

In [12]:
bookid_title = df["bookID"].iloc[1847].split('/')


# content of the remaining columns
remaining_columns = df[['title', 'authors', 'average_rating', 'isbn', 'isbn13',
                                   'language_code', 'num_pages', 'ratings_count', 
                                   'text_reviews_count', 'publication_date']].iloc[1847]


# rearrange cells content for each column
df.iloc[1847, 2:] = remaining_columns
df["bookID"].iloc[1847]= bookid_title[0]
df["title"].iloc[1847]= bookid_title[1]

C:\Users\cleme\AppData\Local\Temp\ipykernel_26336\3963356864.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["bookID"].iloc[1847]= bookid_title[0]
C:\Users\cleme\AppData\Local\Temp\ipykernel_26336\3963356864.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["title"].iloc[1847]= bookid_title[1]


In [13]:
# rechek the content 
df.iloc[1847]

bookID                                          6549
title                  said the shotgun to the head.
authors                                Saul Williams
average_rating                                  4.22
isbn                                     743470796.0
isbn13                                 9780743470797
language_code                                  en-US
num_pages                                        192
ratings_count                                   2762
text_reviews_count                               214
publication_date                            9/1/2003
publisher                                  MTV Books
Name: 1847, dtype: object

In [14]:
# Check the types of the columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   bookID              11127 non-null  object
 1   title               11127 non-null  object
 2   authors             11127 non-null  object
 3   average_rating      11127 non-null  object
 4   isbn                11127 non-null  object
 5   isbn13              11127 non-null  object
 6   language_code       11127 non-null  object
 7   num_pages           11127 non-null  object
 8   ratings_count       11127 non-null  int64 
 9   text_reviews_count  11127 non-null  object
 10  publication_date    11127 non-null  object
 11  publisher           11127 non-null  object
dtypes: int64(1), object(11)
memory usage: 1.0+ MB


For some reasons the supposedly numerical columns became objects 
Re-investigate again if they contain non-numerical content

In [15]:
# re-investigate if there is non-numerical content in supposedly numerical columns
columns = ["average_rating", "isbn13", "text_reviews_count", "num_pages", "ratings_count", "text_reviews_count"]
check_non_numerical_date(columns, "numerical")

The non_numerical_values in column average_rating :  Series([], Name: average_rating, dtype: object)
The non_numerical_values in column isbn13 :  Series([], Name: isbn13, dtype: object)
The non_numerical_values in column text_reviews_count :  Series([], Name: text_reviews_count, dtype: object)
The non_numerical_values in column num_pages :  Series([], Name: num_pages, dtype: object)
The non_numerical_values in column ratings_count :  Series([], Name: ratings_count, dtype: int64)
The non_numerical_values in column text_reviews_count :  Series([], Name: text_reviews_count, dtype: object)


They do not contain non-numerical content, so will transform them to numerical

In [16]:
# convert to numerical
def convert_to_numerical(columns_list: list):
    
    for column in columns_list:
        df[column] = df[column].apply(pd.to_numeric)

columns = ["average_rating", "isbn13", "text_reviews_count", "num_pages", "ratings_count", "text_reviews_count"]

convert_to_numerical(columns)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bookID              11127 non-null  object 
 1   title               11127 non-null  object 
 2   authors             11127 non-null  object 
 3   average_rating      11127 non-null  float64
 4   isbn                11127 non-null  object 
 5   isbn13              11127 non-null  int64  
 6   language_code       11127 non-null  object 
 7   num_pages           11127 non-null  int64  
 8   ratings_count       11127 non-null  int64  
 9   text_reviews_count  11127 non-null  int64  
 10  publication_date    11127 non-null  object 
 11  publisher           11127 non-null  object 
dtypes: float64(1), int64(4), object(7)
memory usage: 1.0+ MB


In [17]:
# fix the dates by replacing 31 with 30 in June and November
# convert the date column to type pd.datetime
df["publication_date"].iloc[8180] = '11/30/2000'
df["publication_date"].iloc[11098] = '6/30/1982'

C:\Users\cleme\AppData\Local\Temp\ipykernel_26336\3398859105.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["publication_date"].iloc[8180] = '11/30/2000'
C:\Users\cleme\AppData\Local\Temp\ipykernel_26336\3398859105.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["publication_date"].iloc[11098] = '6/30/1982'


Resave the DataFrame to avoid repeating the process each time we want to use the df

In [18]:
df.to_csv("books_updated.csv", index_label=False) # index_label = False so that it does not add another index label to the DataFrame

# Data Cleaning Phase 2

This data cleaning is done on the data set after doing webscraping. Through webscraping extra features were extracted which are: 

- first_publish : this is the date a book was first published (this will help distinguish two different books with the same name)
- book_format : this is the format of the book (some books are present in different formats (paperback, Audio CD, hardcover))
- new_publisher: this is because it was noticed that in the original data some books had wrong publishers
- edition_avgRating: this is the actaul average rating of each edition
- added_toShelves: this is the number of users that added a book to shelves

*For more details about the process of scraping, please refer to scraper/scraper.py*

### Read the Data

In [463]:
# read the scraped data
df_scraped = pd.read_csv("scraper/booksRating_extraFeats.csv")

In [464]:
df_scraped.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,first_published,book_format,new_publisher,edition_avgRating,added_toShelves
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,2006-09-16,Scholastic Inc.,"July 16, 2005",Paperback,Scholastic Inc,4.57,4405980.0
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,2004-09-01,Scholastic Inc.,"June 21, 2003",Paperback,Scholastic Inc.,4.50,4518536.0
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,2003-11-01,Scholastic,"July 2, 1998",Hardcover,NaN,4.05,7469.0
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,2004-05-01,Scholastic Inc.,"July 8, 1999",Mass Market Paperback,Scholastic Inc.,4.57,5223956.0
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,2004-09-13,Scholastic,"October 1, 2003",Paperback,Scholastic,4.72,172736.0


In [465]:
df_scraped.tail()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,first_published,book_format,new_publisher,edition_avgRating,added_toShelves
11122,45631,Expelled from Eden: A William T. Vollmann Reader,William T. Vollmann/Larry McCaffery/Michael He...,4.06,1560254416,9781560254416,eng,512,156,20,2004-12-21,Da Capo Press,"December 1, 2004",Paperback,Da Capo Press,4.06,552.0
11123,45633,You Bright and Risen Angels,William T. Vollmann,4.08,0140110879,9780140110876,eng,635,783,56,1988-12-01,Penguin Books,"June 27, 1987",Paperback,Penguin Books,4.04,4269.0
11124,45634,The Ice-Shirt (Seven Dreams #1),William T. Vollmann,3.96,0140131965,9780140131963,eng,415,820,95,1993-08-01,Penguin Books,"January 1, 1990",Paperback,Penguin Publishing Group,3.98,4686.0
11125,45639,Poor People,William T. Vollmann,3.72,0060878827,9780060878825,eng,434,769,139,2007-02-27,Ecco,"January 1, 2007",Hardcover,Ecco,3.75,2948.0
11126,45641,Las aventuras de Tom Sawyer,Mark Twain,3.91,8497646983,9788497646987,spa,272,113,12,2006-05-28,Edimat Libros,"June 1, 1876",Paperback,NaN,3.72,228.0


In [466]:
df_scraped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bookID              11127 non-null  int64  
 1   title               11127 non-null  object 
 2   authors             11127 non-null  object 
 3   average_rating      11127 non-null  float64
 4   isbn                11127 non-null  object 
 5   isbn13              11127 non-null  int64  
 6   language_code       11127 non-null  object 
 7   num_pages           11127 non-null  int64  
 8   ratings_count       11127 non-null  int64  
 9   text_reviews_count  11127 non-null  int64  
 10  publication_date    11127 non-null  object 
 11  publisher           11127 non-null  object 
 12  first_published     11125 non-null  object 
 13  book_format         11126 non-null  object 
 14  new_publisher       8360 non-null   object 
 15  edition_avgRating   11124 non-null  float64
 16  adde

### Fill Missing Data

In the new dataset, there is 2 missing data in the first_published, 1 missing value in the book_format, 3 missing values in the edition_avgRating and added_toShelves. These were values that the scraper failed to get. Since they are very few, they will be filled manually. 

In [467]:
df_scraped[df_scraped[["first_published", "book_format", "edition_avgRating", "added_toShelves"]].isna().any(axis=1)]

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,first_published,book_format,new_publisher,edition_avgRating,added_toShelves
2255,8077,Animales No Se Visten Los (Animals Should Def...,Judi Barrett/Ron Barrett,4.11,1595191356,9781595191359,eng,32,0,0,1991-10-30,Live Oak Media,"January 1, 1970",Paperback,Atheneum Books for Young Readers,NaN,NaN
6396,24062,The Deep (Dive Trilogy),Gordon Korman,3.78,0613674839,9780613674836,eng,148,0,0,2003-07-01,Turtleback Books,NaN,NaN,NaN,NaN,NaN
8476,32552,Essential Tales and Poems,Edgar Allan Poe/Benjamin F. Fisher,4.36,1593080646,9781593080648,en-US,688,66382,109,2004-10-25,Barnes Noble Classics,NaN,Paperback,Barnes & Noble Classics,4.36,92622.0
8520,32703,The Diary of Ellen Rimbauer: My Life at Rose Red,Joyce Reardon/Steven Rimbauer/Ridley Pearson,3.67,0786890436,9780786890439,eng,277,7852,352,2001-04-29,Hyperion,"January 1, 2001",Mass Market Paperback,Hyperion,NaN,NaN


In [468]:
df_scraped.loc[2255, ["edition_avgRating", "added_toShelve"]] = [4.11, 5402]
# df_scraped.loc[6396, ["first_published", "book_format", "new_publisher", "edition_avgRating", "added_toShelve"]] ### !!!! Couldn'y find exact book edition online !!!!
df_scraped.loc[8476, ["first_published"]] = ["January 1, 1843"]
df_scraped.loc[8520, ["edition_avgRating", "added_toShelve"]] = [3.7, 15942]

One of the books at index 6396 was not found on good reads even when searching by book ID, isbn and isbn13. Therefore, the edition_avgRating will be the same as the average_rating. added_toShelves will be equal to the mean of the added_toShelves of the other editions of the book; if other editions are not found the added_toShelves will be equal to the mode of the books having same ratings_count and text_reviews_count. The first_published will be the same as the publication_date and the book_format will be the mode of the books of approximately the same size. 

In [469]:
df_scraped[df_scraped['title'] == df_scraped.loc[6396, 'title']]

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,first_published,book_format,new_publisher,edition_avgRating,added_toShelves,added_toShelve
6396,24062,The Deep (Dive Trilogy),Gordon Korman,3.78,0613674839,9780613674836,eng,148,0,0,2003-07-01,Turtleback Books,NaN,NaN,NaN,NaN,NaN,NaN


In [470]:
df_scraped.loc[df_scraped['title'].str.contains('dive', case=False)]

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,first_published,book_format,new_publisher,edition_avgRating,added_toShelves,added_toShelve
2021,7271,Marine Conservation Biology: The Science of Ma...,Elliott A. Norse/Larry B. Crowder/Michael E. S...,4.59,1559636629,9781559636629,eng,496,16,2,2005-05-09,Island Press,"May 9, 2005",Paperback,Island Press,4.50,175.0,NaN
4458,16059,The Dive From Clausen's Pier,Ann Packer,3.42,0375727132,9780375727139,eng,432,19844,1747,2003-04-08,Vintage,"January 1, 2002",Paperback,Vintage,3.44,34745.0,NaN
4459,16063,The Dive from Clausen's Pier,Ann Packer,3.42,0749933631,9780749933630,eng,368,60,8,2003-03-27,Piatkus,"January 1, 2002",Paperback,NaN,3.15,90.0,NaN
6385,24037,The Deep (Dive #2),Gordon Korman,3.78,0439507235,9780439507233,eng,148,1436,55,2003-07-01,Scholastic,"July 1, 2003",Paperback,Scholastic,3.78,2837.0,NaN
6386,24040,The Discovery (Dive #1),Gordon Korman,3.72,0439507227,9780439507226,eng,141,1858,135,2003-06-01,Apple Paperbacks (Scholastic),"September 1, 2005",Paperback,Scholastic,3.71,3896.0,NaN
6387,24043,New York City's Best Dive Bars: Drinking and D...,Wendy Mitchell/June Kim,3.75,0970312539,9780970312532,eng,160,16,2,2003-07-01,Gamble Guides,"December 1, 2002",Paperback,Gamble Guides,3.71,32.0,NaN
6396,24062,The Deep (Dive Trilogy),Gordon Korman,3.78,0613674839,9780613674836,eng,148,0,0,2003-07-01,Turtleback Books,NaN,NaN,NaN,NaN,NaN,NaN


-- Clemence --

I found the book on GoodReads : https://www.goodreads.com/book/show/24037.The_Deep

As the information on the goodreads page are not exactly the same as the ones in line 6385, I filled the information manually based on the GoddReads page (the title, num_pages and average rating where the same in the database and the link to goodread).

In [471]:
df_scraped.loc[6396, ["ratings_count", "text_reviews_coun", "first_published","book_format","new_publisher","edition_avgRating","added_toShelves"]] = [1659,66,"July 1, 2003","Paperback","Scholastic",3.78,3004]

In [472]:
df_scraped.iloc[6396]

bookID                                  24062
title                 The Deep (Dive Trilogy)
authors                         Gordon Korman
average_rating                           3.78
isbn                               0613674839
isbn13                          9780613674836
language_code                             eng
num_pages                                 148
ratings_count                            1659
text_reviews_count                          0
publication_date                   2003-07-01
publisher                    Turtleback Books
first_published                  July 1, 2003
book_format                         Paperback
new_publisher                      Scholastic
edition_avgRating                        3.78
added_toShelves                        3004.0
added_toShelve                            NaN
text_reviews_coun                        66.0
Name: 6396, dtype: object

For the new_publisher, the scraper failed to get a lot of values, this is because of how the publisher is saved in the html. Due to constraint of time, and since there is already a publisher column in the original dataset. In the cases where there are NaN the old publisher will be put. The new publisher will be taken, only in the cases where the new publisher is different from the old publisher in value.

In [473]:
# Let's create the function for it. This function will only be use when the text format will be editing, 
#in the subtitle "publisher"

def update_publishers(df):
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Check if the new publisher is different from the old publisher
        if pd.notna(row['new_publisher']) and row['new_publisher'] != row['publisher']:
            # Update the publisher with the new publisher value
            df.at[index, 'publisher'] = row['new_publisher']
        else:
            # Use the old publisher if the new publisher is NaN or the same as the old publisher
            df.at[index, 'publisher'] = row['publisher']
    return df

### Set text columns to lower text

When dealing with text data it always safer to have them all in the same case either lower or upper 

In [474]:
def to_lower(df: pd.DataFrame, columns: list):
    """Function to convert text columns to lowercase"""
    for col in columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.lower()
    return df

In [475]:
# get the text columns
text_columns = df_scraped.select_dtypes(include=['object']).columns

# Convert to lowercase
df_scraped = to_lower(df_scraped, text_columns)

In [476]:
df_scraped.sample(5)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,first_published,book_format,new_publisher,edition_avgRating,added_toShelves,added_toShelve,text_reviews_coun
2609,9585,halbschlaf im froschpyjama,tom robbins/pociao/walter hartmann,3.74,3499224429,9783499224423,ger,459,22,2,1998-11-02,rowohlt verlag,"january 1, 1994",paperback,NaN,3.42,40.0,NaN,NaN
4009,14249,prodigal summer,barbara kingsolver,4.00,0060959037,9780060959036,eng,444,88241,5652,2001-10-16,harper perennial,"october 17, 2000",paperback,harper perennial,4.02,186478.0,NaN,NaN
6556,24729,after the plague: and other stories,t. coraghessan boyle,3.86,0142001414,9780142001417,eng,303,1253,108,2002-12-31,penguin books,"september 10, 2001",paperback,penguin publishing group,3.88,2554.0,NaN,NaN
3666,13254,name dropping,jane heller,3.52,0312978332,9780312978334,eng,323,701,53,2001-03-15,st. martin's press,"april 28, 2000",mass market paperback,st. martin's paperbacks,3.51,1450.0,NaN,NaN
2946,10910,mercy,jodi picoult,3.58,0743422449,9780743422444,eng,400,42046,2564,2001-04-01,washington square press,"january 1, 1996",paperback,atria/emily bestler books,3.58,96944.0,NaN,NaN


### Take First Author Name and Create num_contributors Column

We have decided to take only the name of the first author, since usually first authors are the main authors. Another column will be added to indicate the total number of authors/contributors to the book.

In [477]:
# take the first author in the authors column
df_scraped["first_author"] = df_scraped["authors"].apply(lambda x: x.split("/")[0].strip())


In [478]:
df_scraped["first_author"].sample(5)

10704         sara gruen
5883      yvon chouinard
6551       sue monk kidd
4985      p.g. wodehouse
2747     margaret hodges
Name: first_author, dtype: object

In [479]:
df_scraped["num_contributors"] = df_scraped["authors"].apply(lambda x: len(x.split("/")))

In [480]:
df_scraped[["authors","num_contributors"]].sample(5)

,authors,num_contributors
1213,amy sedaris,1
10496,roddy doyle,1
1995,cesare beccaria/david young,2
10685,t.h. white,1
4684,f. scott fitzgerald,1


### Clean Text Columns

Remove punctuation, and extra white spaces from all text columns.

Better clean the text in publisher column, to avoid having same publishers represented differently. 
Example: indexes 5993, 5365 and 9753, W.W. Norton & Company, W. W. Norton and Company and W. W. Norton  Company represent the same publisher

In [481]:
general_replacements = [
    (r'[^\w\s]', ''), # remove punctuation
    (r'\s\s+', " "), # remove double spaces and more   
]

columns_ = ["title", "first_author"]

df_scraped = sub_text(df_scraped, columns_, general_replacements)

In [482]:
df_scraped[["title", "first_author"]].sample(5)

,title,first_author
2984,eight men out the black sox and the 1919 world...,eliot asinof
7160,waterfront a walk around manhattan,phillip lopate
211,tesla papers,nikola tesla
9665,vegas bites a werewolf romance anthology,la banks
194,pyramids of montauk explorations in consciousness,peter moon


In [483]:
specific_replacements = [
    ("inc.", ""),
    ("llc", ""),
    ("ltd", ""),
    ("w. w.", "ww"), 
    ("&", " and "),
    (r'\bbooks?\b', ""),  
    (r'\bclassics?\b', ""),
    (r'\bpublishers?\b', ""),
    (r'\bpress\b', ""),
    (r'\bpublishing\b', "")
]

df_scraped = sub_text(df_scraped, ["publisher"], specific_replacements)

### Update publisher column with "new_publisher"

In [484]:
#Use update_publishers function to be sure about our "publisher" column : 

In [485]:
df_scraped[['publisher', 'new_publisher']].tail(50)

,publisher,new_publisher
11077,debols!llo,NaN
11078,montena,NaN
11079,montena,NaN
11080,listening library,NaN
11081,listening library (audio),listening library (audio)
11082,alfred a. knopf for young readers,NaN
11083,alfred a. knopf for young readers,NaN
11084,listening library,NaN
11085,alfred a. knopf,NaN
11086,ediciones b,NaN


-- Clemence --

Based on the sample, i'm not sure we should always take the new_publisher column... For example for line 11121, 11123 and 11124, with the "publisher" column we have the same publisher : Penguin books. With the new_publisher column we will have 2 different publishers (Penguin Publishing Group and Penbguin Books). I don't remember why you wanted to add new information about publisher ? There was trouble with the publisher column ? Otherwise I propose to work only on the publisher column if it's okay with you :) I did this below

### Add "size_of_publisher" column

In [486]:
# Adding the new columns that allows to see if a publisher is often cited
df_scraped['publisher_count'] = df_scraped.groupby('publisher')['publisher'].transform('count')

In [487]:
# The exact number of time a publisher name is in the database is not as relevant as the number of time an author or a title is
# in it.
# To simplify this information, we can, instead of the count, create a column that categorize the publisher :
# - Small publisher(1) = only 1 time in the database
# - Medium publisher(2) = from 2 to 10 times
# - big publisher(3) = more that 10 times

# Create a function to determine the size of the publisher based on the number of times it is mentioned
def determine_size(publisher_count):
    if publisher_count == 1:
        return 1
    elif publisher_count < 10:
        return 2
    else:
        return 3

# Apply the function to the 'publisher_count' column to create the new 'size_of_publisher' column
df_scraped['size_of_publisher'] = df_scraped['publisher_count'].apply(lambda x: determine_size(x))

In [488]:
print('Number of line with small publisher',len(df_scraped[df_scraped['size_of_publisher'] == 1]))
print('Number of line with medium publisher',len(df_scraped[df_scraped['size_of_publisher'] == 2]))
print('Number of line with big publisher',len(df_scraped[df_scraped['size_of_publisher'] == 3]))

Number of line with small publisher 1254
Number of line with medium publisher 2647
Number of line with big publisher 7226


In [489]:
#Now that we have the "size_of_publisher" column, let's delete the "publisher_count" used to create it 
df_scraped.drop(columns=['publisher_count'], inplace=True)

### Create "num_book_per_author" column

In [490]:
# Check for duplicates in a specific column using value_counts
num_book_per_author = df_scraped["first_author"].value_counts()

# Display values with count greater than 1 (indicating duplicates)
print(num_book_per_author[num_book_per_author > 1])

william shakespeare    88
stephen king           82
jrr tolkien            51
pg wodehouse           46
agatha christie        45
                       ..
paul farmer             2
amy sedaris             2
ruby ann boxcar         2
anthony loyd            2
james lee burke         2
Name: first_author, Length: 1471, dtype: int64


In [491]:
# Adding the new columns that allows to see if an author is often cited
df_scraped['num_book_per_author'] = df_scraped.groupby('first_author')['first_author'].transform('count')

In [492]:
df_scraped.sample(10)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,...,book_format,new_publisher,edition_avgRating,added_toShelves,added_toShelve,text_reviews_coun,first_author,num_contributors,size_of_publisher,num_book_per_author
4419,15888,an instance of the fingerpost,iain pears,3.94,1573227951,9781573227957,eng,691,19134,1081,...,paperback,penguin publishing group,3.93,46014.0,NaN,NaN,iain pears,1,3,4
10845,44254,la telaraña de carlota,e.b. white/guillermo solana/garth williams,4.17,006075740x,9780060757403,spa,224,225,38,...,paperback,NaN,4.18,1041.0,NaN,NaN,eb white,3,2,11
4418,15887,la historia de la familia roccamatio de helsinki,yann martel/bianca southwood,3.48,8423338665,9788423338665,spa,91,14,2,...,hardcover,NaN,3.05,38.0,NaN,NaN,yann martel,2,1,4
8868,34298,dublin 4,maeve binchy,3.64,0099498588,9780099498582,en-us,256,2155,59,...,paperback,arrow,3.65,5540.0,NaN,NaN,maeve binchy,1,3,18
8184,31386,ariel sharon a life,nir hefez/gadi bloom/mitch ginsburg,3.72,1400065879,9781400065875,en-us,490,40,6,...,hardcover,"random house, inc.",3.73,112.0,NaN,NaN,nir hefez,3,2,1
7014,26590,the final battle tales from the odyssey 6,mary pope osborne/troy howell,4.08,0786809949,9780786809943,en-us,112,263,17,...,paperback,hyperion,4.12,712.0,NaN,NaN,mary pope osborne,2,3,12
2471,9006,dont get too comfortable the indignities of co...,david rakoff,3.72,0767916034,9780767916035,eng,222,8410,751,...,paperback,anchor,3.73,21973.0,NaN,NaN,david rakoff,1,3,1
8959,34762,legends lies cherished myths of world history,richard shenkman,3.47,0060922559,9780060922559,eng,320,222,16,...,paperback,william morrow paperbacks,3.46,520.0,NaN,NaN,richard shenkman,1,3,1
4852,17473,black beauty coloring book,anna sewell/john green,3.96,048629272x,9780486292724,eng,48,68,3,...,paperback,dover publications,4.13,151.0,NaN,NaN,anna sewell,2,3,3
3642,13149,the thomas berryman number,james patterson,2.86,0446600458,9780446600453,eng,272,4320,307,...,paperback,grand central publishing,2.85,11481.0,NaN,NaN,james patterson,1,3,43


### Create "is_english" column

In [493]:
# Add a new column 'is_english' with 1 for English (en, en-CA, en-GB,...) and 0 for non-English
df_scraped['is_english'] = np.where(df_scraped['language_code'].str.contains('en', case=False), 1, 0)

In [494]:
df_scraped.sample(10)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,...,new_publisher,edition_avgRating,added_toShelves,added_toShelve,text_reviews_coun,first_author,num_contributors,size_of_publisher,num_book_per_author,is_english
8198,31466,far from the madding crowd,thomas hardy,3.94,0140620478,9780140620474,en-gb,374,320,42,...,NaN,3.77,917.0,NaN,NaN,thomas hardy,1,3,10,1
6330,23846,mischief harrow house 2,douglas clegg,3.49,0843947667,9780843947663,eng,359,385,29,...,leisure books,3.40,1008.0,NaN,NaN,douglas clegg,1,3,9,1
494,1559,oedipus the king,sophocles/bernard knox/cynthia brantley johnson,3.70,1416500332,9781416500339,eng,144,740,57,...,NaN,3.62,1588.0,NaN,NaN,sophocles,3,3,19,1
11034,45280,quicksilver the baroque cycle 1,neal stephenson,3.93,0434008176,9780434008179,eng,927,171,12,...,NaN,4.08,355.0,NaN,NaN,neal stephenson,1,1,14,1
9129,35515,the inner reaches of outer space metaphor as m...,joseph campbell,4.22,1577312090,9781577312093,en-gb,160,971,49,...,new world library,4.22,3512.0,NaN,NaN,joseph campbell,1,2,3,1
4284,15407,the lord of the rings millennium edition boxed...,j.r.r. tolkien,4.50,0618037667,9780618037667,eng,1472,120,5,...,NaN,4.80,192.0,NaN,NaN,jrr tolkien,1,3,51,1
9433,37420,the short stories of ernest hemingway,ernest hemingway,4.26,0020518609,9780020518600,eng,499,113,12,...,NaN,3.89,422.0,NaN,NaN,ernest hemingway,1,2,18,1
9300,36569,ice blue ice 3,anne stuart,3.94,0778324788,9780778324782,eng,378,3215,197,...,mira,3.93,6017.0,NaN,NaN,anne stuart,1,3,10,1
6543,24671,the tristan betrayal,robert ludlum,3.84,0312990685,9780312990688,eng,505,5942,150,...,st. martin's paperbacks,3.86,10694.0,NaN,NaN,robert ludlum,1,3,10,1
4275,15351,the return of the shadow the history of the lo...,j.r.r. tolkien/christopher tolkien,4.03,061808357x,9780618083572,eng,512,2343,92,...,houghton mifflin,4.07,14574.0,NaN,NaN,jrr tolkien,2,3,51,1


### Create "book_count" column

In [495]:
# Adding the new columns that allows to see if a book is often cited
df_scraped['book_count'] = df_scraped.groupby('title')['title'].transform('count')

In [496]:
df_scraped.query("title == 'the brothers karamazov'")

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,...,edition_avgRating,added_toShelves,added_toShelve,text_reviews_coun,first_author,num_contributors,size_of_publisher,num_book_per_author,is_english,book_count
1406,4933,the brothers karamazov,fyodor dostoyevsky/constance garnett/manuel ko...,4.32,0451527348,9780451527349,eng,736,983,91,...,4.27,1638.0,NaN,NaN,fyodor dostoyevsky,4,3,37,1,9
1407,4934,the brothers karamazov,fyodor dostoyevsky/fyodor dostoevsky/richard p...,4.32,0374528373,9780374528379,eng,796,191531,6795,...,4.35,855582.0,NaN,NaN,fyodor dostoyevsky,4,3,37,1,9
1408,4935,the brothers karamazov,fyodor dostoyevsky/david mcduff,4.32,0140449248,9780140449242,eng,1013,1673,184,...,4.44,11774.0,NaN,NaN,fyodor dostoyevsky,2,3,37,1,9
1409,4936,the brothers karamazov,fyodor dostoyevsky/richard pevear/larissa volo...,4.32,0679729259,9780679729259,eng,796,617,80,...,4.51,1381.0,NaN,NaN,fyodor dostoyevsky,3,3,37,1,9
1410,4938,the brothers karamazov,fyodor dostoyevsky/simon vance/thomas r. beyer...,4.32,1596440791,9781596440791,eng,16,20,2,...,4.52,164.0,NaN,NaN,fyodor dostoyevsky,3,2,37,1,9
1411,4940,the brothers karamazov,fyodor dostoyevsky/constance garnett/maire jaanus,4.32,159308045x,9781593080457,eng,720,1089,202,...,4.38,3421.0,NaN,NaN,fyodor dostoyevsky,3,3,37,1,9
1634,5691,the brothers karamazov,fyodor dostoyevsky/richard pevear/larissa volo...,4.32,0099922800,9780099922803,eng,796,443,55,...,4.45,1657.0,NaN,NaN,fyodor dostoyevsky,3,3,37,1,9
1990,7135,the brothers karamazov,fyodor dostoyevsky/andrew r. macandrew/konstan...,4.32,0553212168,9780553212167,eng,1072,1022,154,...,4.37,3236.0,NaN,NaN,fyodor dostoyevsky,3,3,37,1,9
9356,37058,the brothers karamazov,fyodor dostoyevsky/thomas r. beyer jr./simon v...,4.32,1596440783,9781596440784,eng,16,3,1,...,4.67,9.0,NaN,NaN,fyodor dostoyevsky,3,2,37,1,9


### Create a "is_serie" column

In [497]:
# Create a regular expression pattern to match titles containing numbers
pattern = r'\b\d+\b'  # This pattern matches one or more digits

# Use the str.contains() method with the regex pattern to filter the DataFrame
books_with_number = df_scraped[df_scraped['title'].str.contains(pattern)]

# Print or further process the extracted books
books_with_number['title'].sample(20)

7014             the final battle tales from the odyssey 6
3815                               tehanu earthsea cycle 4
6401                            casino royale james bond 1
5907                the princess bride long tall texans 15
705      murder at the washington tribune capital crime...
10474                 nimitz class admiral arnold morgan 1
7263     algorithms in c parts 14 fundamentals data str...
9192                on basilisk station honor harrington 1
5701                               already dead joe pitt 1
3137                    1 000 places to see before you die
8923               lords and ladies discworld 14 witches 4
963                               greek tragedies volume 2
7006        night of blood dragonlance the minotaur wars 1
8310                   astonishing xmen volume 2 dangerous
827                             the lost boy dave pelzer 2
11063                          schmidt delivered schmidt 2
6079                                                犬夜叉 

In [498]:
# Add a new column 'is_serie' with 1 for books with a number in the title and 0 for books without numbers
df_scraped['is_serie'] = np.where(df_scraped['title'].str.contains(pattern), 1, 0)

In [499]:
df_scraped.sample(10)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,...,added_toShelves,added_toShelve,text_reviews_coun,first_author,num_contributors,size_of_publisher,num_book_per_author,is_english,book_count,is_serie
6753,25347,out of egypt christ the lord 1,anne rice,3.58,0345436830,9780345436832,eng,350,11519,981,...,21868.0,NaN,NaN,anne rice,1,3,23,1,1,1
5261,18981,hayao miyazaki master of japanese animation,helen mccarthy,3.93,1880656418,9781880656419,en-us,240,317,19,...,1199.0,NaN,NaN,helen mccarthy,1,2,1,1,1,0
3620,13113,zen and the art of motorcycle maintenance an i...,robert m. pirsig,3.77,0688052304,9780688052300,eng,412,424,47,...,1155.0,NaN,NaN,robert m pirsig,1,2,4,1,1,0
1115,3768,book of sketches,jack kerouac/george condo,3.80,0142002151,9780142002155,eng,432,572,43,...,1576.0,NaN,NaN,jack kerouac,2,3,11,1,1,0
8497,32627,everville book of the art 2,clive barker,4.03,0060933151,9780060933159,eng,697,8829,157,...,17013.0,NaN,NaN,clive barker,1,2,11,1,1,1
10002,40289,the abyss,orson scott card/james francis cameron,4.07,0099690608,9780099690603,eng,363,17911,111,...,30412.0,NaN,NaN,orson scott card,2,1,40,1,1,0
9727,38830,stranger from the past,penny jordan,3.23,0373115997,9780373115990,eng,224,29,7,...,73.0,NaN,NaN,penny jordan,1,2,2,1,1,0
3930,13996,alta dragon jousters 2,mercedes lackey,4.03,0756402573,9780756402570,en-us,434,7060,113,...,11224.0,NaN,NaN,mercedes lackey,1,3,40,1,1,1
8492,32592,in wonderland,knut hamsun/sverre lyngstad,3.60,0970312555,9780970312556,nor,185,86,8,...,342.0,NaN,NaN,knut hamsun,2,1,7,0,1,0
7023,26653,the coal tattoo,silas house,4.04,0345480058,9780345480057,eng,368,1789,144,...,3855.0,NaN,NaN,silas house,1,3,3,1,1,0


In [500]:
#Let's see if we have to add some books in "is_serie" based on word in there title
df_scraped[(df_scraped['is_serie'] == '0') & (df_scraped['title'].str.contains('trilogy|tome|chronicles|series', case=False))]

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,...,added_toShelves,added_toShelve,text_reviews_coun,first_author,num_contributors,size_of_publisher,num_book_per_author,is_english,book_count,is_serie


### Convert the first_published date into 3 columns : published_year, published_month, published_day

By looking at the data, we notice that for some line, the first_published date didn't extracted well, and it's written "ary" instead of january or february.

In [501]:
df_scraped[['title','first_published']][df_scraped['first_published'].str.contains(r'ary', case=False) & ~df_scraped['first_published'].str.contains(r'january|february', case=False)]

,title,first_published
56,simply beautiful beaded jewelry,"ary 28, 2006"
77,the power of one the solo play for playwrights...,"ary 7, 2000"
124,tropic of capricorn,"ary 1, 1939"
176,gravitys rainbow,"ary 28, 1973"
179,gravitys rainbow,"ary 28, 1973"
...,...,...
11067,the home front,"ary 23, 1989"
11090,la conspiración de los alquimistas,"ary 1, 1999"
11092,the call of the mall how we shop,"ary 2, 2004"
11101,undaunted courage the pioneering first mission...,"ary 15, 1996"


In [502]:
#We will replace the "ary" in those line by "february" 
# Filter rows containing "ary" but not "january" or "february"
filtered_rows = df_scraped['first_published'].str.contains(r'ary', case=False) & ~df_scraped['first_published'].str.contains(r'january|february', case=False)

# Replace the filtered results with "february"
df_scraped.loc[filtered_rows, 'first_published'] = df_scraped.loc[filtered_rows, 'first_published'].str.replace(r'ary', 'february', case=False)

In [503]:
df_scraped[['title','first_published']][df_scraped['first_published'].str.contains('published', case=False)]

,title,first_published
133,love letters,"published january 1, 1999"
201,timbuktu leviathan moon palace,"published october 31, 2002"
203,the coming economic collapse how you can thriv...,"published february 21, 2006"
229,guidebook to zen and the art of motorcycle mai...,"published november 19, 1990"
263,best of london lonely planet best of,"published january 1, 2004"
...,...,...
10881,the best american travel writing 2006,"published october 11, 2006"
10898,poems between women four centuries of love rom...,"published april 15, 1999"
10937,un amour de swann à la recherche du temps perd...,"published january 1, 2006"
10947,poetry and prose of alexander pope riverside e...,"published january 2, 1968"


In [504]:
#Let's delete the word "published" from the first_published column
df_scraped['first_published'] = df_scraped['first_published'].str.replace('published ', '')

By trying to convert the "first_published" column into date, we notice several errors due to date before 1677. Indeed, the smallest date that pandas can handle is January 1, 1677. As we are looking for an average rating given on a website, there is probably not a big difference between books written in 1524 or 1678, as they are considered "old". By lack of time, we'll change all year below 1677 by 1678.

In [505]:
# The smallest date that pandas can handle is January 1, 1677. Let's remove all date below 1677 and change the year by 1678
for i, date_str in enumerate(df_scraped['first_published']):
    # Use regular expression to find the year in the string
    match = re.search(r'\b\d{1,4}\b$', date_str)
    if match:
        year = int(match.group())
        if year < 1677:
            # Replace the year with 1678
            modified_date_str = re.sub(r'\b\d{1,4}\b$', '1678', date_str)
            df_scraped.at[i, 'first_published'] = modified_date_str

In [506]:
df_scraped['first_published'] = pd.to_datetime(df_scraped['first_published'], format='%B %d, %Y')

In [507]:
# Convert the first_published date into 3 columns : published_year, published_month, published_day
df_scraped['published_year'] = df_scraped['first_published'].dt.year
df_scraped['published_month'] = df_scraped['first_published'].dt.month
df_scraped['published_day'] = df_scraped['first_published'].dt.day


In [508]:
df_scraped.tail(20)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,...,first_author,num_contributors,size_of_publisher,num_book_per_author,is_english,book_count,is_serie,published_year,published_month,published_day
11107,45572,fantastic mr fox cover to cover,roald dahl,4.05,1855495090,9781855495098,eng,90,7,0,...,roald dahl,1,2,41,1,1,0,1970,9,12
11108,45574,ajax in action,dave crane/eric pascarello/darren james,3.56,1932394613,9781932394610,en-us,680,126,10,...,dave crane,3,2,1,1,1,0,2005,10,31
11109,45583,historia del rey transparente,rosa montero,3.90,8466318771,9788466318778,spa,592,1266,90,...,rosa montero,1,2,2,0,1,0,2005,1,1
11110,45585,bella y oscura,rosa montero,3.66,843221728x,9788432217289,spa,204,293,31,...,rosa montero,1,3,2,0,1,0,1993,1,1
11111,45592,la tía julia y el escribidor,mario vargas llosa,3.92,8466302298,9788466302296,spa,566,162,10,...,mario vargas llosa,1,1,2,0,2,0,1977,3,1
11112,45595,la tía julia y el escribidor,mario vargas llosa,3.92,8432203238,9788432203237,spa,447,25,4,...,mario vargas llosa,1,2,2,0,2,0,1977,3,1
11113,45604,narraciones extraordinarias,edgar allan poe,4.13,9583006408,9789583006401,spa,316,36,4,...,edgar allan poe,1,1,11,0,1,0,1840,1,1
11114,45607,las crónicas de narnia,c.s. lewis/margarita e. valdes/gemma gallart/p...,4.26,0061199001,9780061199004,spa,816,186,11,...,cs lewis,4,2,33,0,1,0,1956,1,1
11115,45615,o trono de prata as crónicas de nárnia 6,c.s. lewis/ana falcão bastos,3.96,972233168x,9789722331685,por,168,141,10,...,cs lewis,2,2,33,0,1,1,1953,9,7
11116,45616,a última batalha as crónicas de nárnia 7,c.s. lewis/pauline baynes/ana falcão bastos,4.03,9722332201,9789722332200,por,149,211,24,...,cs lewis,3,2,33,0,1,1,1956,9,4
